# VOICE AND FACE AUTHENTICATION SYSTEM

### RUN THE CELL ONE BY ONE AFTER READING INSTRUCTION

## Please install necessary library required

In [ ]:
import tensorflow as tf
import numpy as np
import os
import glob
import pickle
import cv2
import time
from numpy import genfromtxt

from keras import backend as K
from tensorflow.keras.models import load_model

K.set_image_data_format('channels_first')
np.set_printoptions(threshold=np.inf)


import pyaudio
from IPython.display import Audio, display, clear_output
import wave
from scipy.io.wavfile import read
#from sklearn.mixture import GMM 
from sklearn.mixture import GaussianMixture 
import warnings
warnings.filterwarnings("ignore")

from sklearn import preprocessing
import python_speech_features as mfcc
import face_recognition
import cv2
import numpy as np

# Audio processing

### -After installling all the necessary library pls run the below cell


In [2]:
def calculate_delta(array):
    rows, cols = array.shape
    deltas = np.zeros((rows, cols))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while len(index) < 2 and j < N+1:
            if i-j >= 0:
                index.append(i-j)
            if i+j < rows:
                index.append(i+j)
            j+=1
        if len(index) == 2:
            deltas[i] = (array[index[1]] - array[index[0]]) / (2*N)
        elif len(index) == 1:
            deltas[i] = (array[index[0]] - array[i]) / N
    return deltas


#convert audio to mfcc features
def extract_features(audio, rate):    
    mfcc_feat = mfcc.mfcc(audio, rate, 0.025, 0.01, 12, appendEnergy=True, nfft=2048)
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)

    #combining both mfcc features and delta
    combined = np.hstack((mfcc_feat, delta)) 
    return combined

# Registering  New User voice

### Run the below code for registering new voice of the person .

### WARNING:- RECORD IN SILENCE


##### -------------------- follow steps below --------------------------------------------

#### - RUN THE CODE 
#### - ENTER  YOUR NAME 
#### - SPEAK HELLO COMPUTER WHEN RECORDING
#### - SPEAK SAME WORD 2 TIMES MORE
#### - NOW YOUR VOICE REGISTERED 

#### - REGISTER THE 2 OR 3 PERSON VOICE 
#### - NOW YOUR VOICE REGISTERED 

###### -----------------------------------------------------------------------------------------------------------------

## TIPS:- register in silence




In [ ]:


# Cell 2: Import all dependencies
import os
import time
import wave
import pickle
import numpy as np
import pyaudio
from sklearn.mixture import GaussianMixture
from IPython.display import clear_output
from scipy.io import wavfile
import python_speech_features
from IPython.display import Audio

# Cell 3: Configuration
FORMAT = pyaudio.paInt16
CHANNELS = 1  # Changed to 1 channel for better compatibility
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 3
PHRASE = "HELLO COMPUTER"

# Cell 4: Audio recording function
def record_audio_sample():
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                       rate=RATE, input=True,
                       frames_per_buffer=CHUNK)
    
    print("Recording...")
    frames = []
    
    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK, exception_on_overflow=False)
        frames.append(data)
    
    stream.stop_stream()
    stream.close()
    audio.terminate()
    return b''.join(frames)

# Cell 5: Feature extraction (fixed version)
def extract_features(audio_path):
    try:
        (rate, sig) = wavfile.read(audio_path)
        mfcc_feat = python_speech_features.mfcc(sig, rate, winlen=0.025, winstep=0.01, numcep=13,
                                                nfilt=26, nfft=2048, lowfreq=0, highfreq=None, preemph=0.97,
                                                ceplifter=22, appendEnergy=True)
        return mfcc_feat
    except Exception as e:
        print(f"Feature extraction failed: {str(e)}")
        return None

# Cell 6: Main registration function
def register_voice():
    name = input("Enter your name: ").strip().lower()
    os.makedirs("./voice_database", exist_ok=True)
    os.makedirs("./gmm_models", exist_ok=True)
    
    source = "./voice_database/" + name
    os.makedirs(source, exist_ok=True)
    
    for i in range(3):
        # Countdown for first sample
        if i == 0:
            for j in range(3, 0, -1):
                clear_output(wait=True)
                print(f"Speak '{PHRASE}' in {j} seconds...")
                time.sleep(1)
        
        # Record audio
        audio_data = record_audio_sample()
        
        # Save recording
        filename = f"{source}/{i+1}.wav"
        with wave.open(filename, 'wb') as wf:
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(pyaudio.get_sample_size(FORMAT))
            wf.setframerate(RATE)
            wf.writeframes(audio_data)
        
        # Verify recording
        try:
            with wave.open(filename, 'rb') as wf:
                if wf.getnframes() == 0:
                    print("Empty recording - please try again")
                    return False
        except:
            print("Invalid recording - please try again")
            return False
    
    # Process recordings
    features = np.array([])
    for i in range(1, 4):
        path = f"{source}/{i}.wav"
        vector = extract_features(path)
        
        if vector is None:
            print(f"Failed to process {path}")
            return False
            
        if features.size == 0:
            features = vector
        else:
            features = np.vstack((features, vector))
    
    # Train model
    gmm = GaussianMixture(n_components=16, max_iter=200, covariance_type='diag', n_init=3)
    gmm.fit(features)
    
    # Save model
    model_path = f"./gmm_models/{name}.gmm"
    with open(model_path, 'wb') as f:
        pickle.dump(gmm, f)
    
    print(f"\nRegistration successful for {name}!")
    return True

# Cell 7: Run registration
register_voice()

# Registering a New User face 

###  Run the below code for registering new face of the person .




##### --------------------------------------------------------follow steps below--------------------------------------------------------------------------------------- 

#### - RUN THE CODE 
#### - ENTER  YOUR NAME 
#### - IT START RECORDING 
#### - PRESS S TO SAVE IMAGE 
#### - AFTER THE MESSAGE SUCCESSFUL PRINTED IN BELOW CELL
#### - PRESS Q TO EXIT
 
#### --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## TIPS:- register 2 to 3 image to help the model to classify 

In [ ]:
import cv2
import os

# Create a directory to store captured images
dir_name = 'captured_images'
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

# Get name of the person to capture images
name = input("Enter your name: ")

# Create a directory with the name of the person
person_dir = os.path.join(dir_name, name)
if not os.path.exists(person_dir):
    os.makedirs(person_dir)

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Set the width and height of the capture window
cap.set(3, 640)
cap.set(4, 480)

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

# Capture frames continuously
while True:
    # Capture a frame
    ret, frame = cap.read()

    # Display instructions in the frame
    cv2.putText(frame, "Press 'q' to exit", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, "Press 's' to save image", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('frame', frame)

    # Exit the camera if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Save the image if 's' is pressed
    if cv2.waitKey(1) & 0xFF == ord('s'):
        # Generate a unique filename for the image
        filename = os.path.join(person_dir, name + "_" + str(len(os.listdir(person_dir))+1) + ".jpg")

        # Save the image
        cv2.imwrite(filename, frame)
        print("Image saved successfully!")

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()




# Automatically add array of face encodings and there name and store it in a empty list


### just run the below code

In [ ]:
import os
import face_recognition
import pickle

def save_user_face_encoding(user_folder_path):
    image_files = [f for f in os.listdir(user_folder_path) if f.lower().endswith(('.jpg', '.png'))]
    if not image_files:
        print(f"No images found in {user_folder_path}")
        return False

    # For better accuracy, you can encode multiple images and average or keep all encodings
    encodings = []
    for img_file in image_files:
        img_path = os.path.join(user_folder_path, img_file)
        image = face_recognition.load_image_file(img_path)
        face_encs = face_recognition.face_encodings(image)
        if face_encs:
            encodings.append(face_encs[0])
        else:
            print(f"No face found in {img_path}")

    if not encodings:
        print(f"No valid face encodings found for {user_folder_path}")
        return False

    # For simplicity, take the first encoding or average encodings
    # Here we take the first encoding
    user_encoding = encodings[0]

    # Save encoding to pickle in user folder
    pickle_path = os.path.join(user_folder_path, 'face_encoding.pkl')
    with open(pickle_path, 'wb') as f:
        pickle.dump(user_encoding, f)

    print(f"Saved face encoding for user at {pickle_path}")
    return True

# Example usage:
base_path = 'captured_images'
for user_name in os.listdir(base_path):
    user_path = os.path.join(base_path, user_name)
    if os.path.isdir(user_path):
        save_user_face_encoding(user_path)




# ----------------------------VOICE RECOGNITION---------------------------

### - JUST SPEAK SAME WORD YOU REGISTERED IN SAME PITCH TO RECOGNISE YOUR VOICE

In [ ]:
import pyaudio
import wave
import os
import pickle
import numpy as np
from scipy.io import wavfile
import python_speech_features

# Configuration parameters (match with registration)
FORMAT = pyaudio.paInt16
CHANNELS = 1  # Changed to 1 to match registration
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 3
FILENAME = "./test.wav"

def extract_features(audio_path):
    """
    Extract MFCC features from a WAV file.
    """
    try:
        (rate, sig) = wavfile.read(audio_path)
        mfcc_feat = python_speech_features.mfcc(sig, rate, winlen=0.025, winstep=0.01, numcep=13,
                                                nfilt=26, nfft=2048, lowfreq=0, highfreq=None, preemph=0.97,
                                                ceplifter=22, appendEnergy=True)
        return mfcc_feat
    except Exception as e:
        print(f"Feature extraction failed: {str(e)}")
        return None

def recognize_voice():
    """
    Record audio from microphone and identify speaker by comparing with saved GMM models.
    """
    audio = pyaudio.PyAudio()

    # Start recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("Recording... please say 'HELLO COMPUTER' for authentication")
    frames = []

    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK, exception_on_overflow=False)
        frames.append(data)
    print("Finished recording.")

    # Stop recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded data as a WAV file
    with wave.open(FILENAME, 'wb') as waveFile:
        waveFile.setnchannels(CHANNELS)
        waveFile.setsampwidth(audio.get_sample_size(FORMAT))
        waveFile.setframerate(RATE)
        waveFile.writeframes(b''.join(frames))

    # Load GMM models
    modelpath = "./gmm_models/"
    gmm_files = [os.path.join(modelpath, fname) for fname in os.listdir(modelpath) if fname.endswith('.gmm')]

    if len(gmm_files) == 0:
        print("No users in the database. Please register first.")
        return None

    models = [pickle.load(open(fname, 'rb')) for fname in gmm_files]
    speakers = [os.path.splitext(os.path.basename(fname))[0] for fname in gmm_files]

    # Extract features from the recorded audio
    vector = extract_features(FILENAME)
    if vector is None:
        print("Could not extract features from the audio.")
        return None

    log_likelihood = np.zeros(len(models))

    # Compute log likelihood for each model
    for i, gmm in enumerate(models):
        scores = gmm.score(vector)
        log_likelihood[i] = scores.sum()

    # Identify the speaker with the highest score
    pred_index = np.argmax(log_likelihood)
    identity = speakers[pred_index]

    print(f"Recognized as: {identity}")
    return identity

# Run the recognition
recognized_user = recognize_voice()
print(f"Authenticated user: {recognized_user}")


                  


# ----------------------------FACE AND VOICE AUTHENTICATION--------------------------

#### - KEEP YOUR FACE INFRONT OF THE CAMERA
### -------------------------------------THE RESULT WILL SHOW AS BELOW-----------------------------------
#### - IF FACE MATCHES WITH VOICE  --Authenticaation successful !welcome-------
#### - IF VOICE IDENTITY AND FACE IDENTIY IS DIFFERENT -----Voice identity not matching with face !Try again....
#### - IF VOICE IDENTITY== UKNOWN     ------------Voice not recognized , !Try again.....
#### - IF FACE IDENTITY == UKNOWN      -------------" Face not registered,! Unsuccessful"


In [ ]:
import os
import cv2
import face_recognition
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output

def load_all_face_encodings(base_path='captured_images'):
    """
    Loads face encodings and user names from individual pickle files stored in user folders.
    """
    known_face_encodings = []
    known_face_names = []

    if not os.path.exists(base_path):
        print(f"Error: Base path '{base_path}' does not exist.")
        return known_face_encodings, known_face_names

    for user_name in os.listdir(base_path):
        user_path = os.path.join(base_path, user_name)
        if not os.path.isdir(user_path):
            continue  # Skip files, only process directories

        pickle_path = os.path.join(user_path, 'face_encoding.pkl')
        if os.path.exists(pickle_path):
            try:
                with open(pickle_path, 'rb') as f:
                    encoding = pickle.load(f)
                known_face_encodings.append(encoding)
                known_face_names.append(user_name)
            except Exception as e:
                print(f"Warning: Could not load encoding for user '{user_name}': {e}")
        else:
            print(f"Warning: No encoding pickle file found for user '{user_name}' in '{user_path}'")

    return known_face_encodings, known_face_names

# === Load all known face encodings and names from user folders ===
known_face_encodings, known_face_names = load_all_face_encodings()

# Identity from voice authentication step (replace with actual voice-authenticated username)

identity = recognize_voice()
print(f"Voice authentication recognized user: {identity}")

print("Keep your face in front of the camera")

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

time.sleep(1.0)
process_this_frame = True

timeout_seconds = 20  # Run for 20 seconds
start_time = time.time()

authenticated = False  # Flag to track if authentication succeeded
last_frame = None     # To store last frame for display

try:
    while True:
        elapsed = time.time() - start_time
        if elapsed > timeout_seconds:
            print("Timeout reached. Ending authentication.")
            break

        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        face_names = []
        face_locations = []

        if process_this_frame:
            face_locations = face_recognition.face_locations(rgb_small_frame)

            if len(face_locations) == 1:
                face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

                for face_encoding in face_encodings:
                    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)

                    if matches[best_match_index]:
                        facename = known_face_names[best_match_index]
                    else:
                        facename = "Unknown"

                    face_names.append(facename)

                    if facename == identity:
                        authenticated = True

            elif len(face_locations) > 1:
                print("More than one face found. Please show only one face.")

        process_this_frame = not process_this_frame

        # Draw rectangles and labels on frame
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX

            if name == identity:
                text = f"{name} Authentication successful! Welcome."
                color = (0, 255, 0)
            elif name == "Unknown":
                text = "Face not registered! Unsuccessful."
                color = (0, 0, 255)
            elif identity == "unknown":
                text = "Voice not recognized! Try again."
                color = (0, 0, 255)
            else:
                text = "Voice and face identity mismatch! Try again."
                color = (0, 0, 255)

            cv2.putText(frame, text, (left + 6, bottom - 6), font, 0.5, color, 1)

        last_frame = frame.copy()

        # Display live frame in notebook
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        clear_output(wait=True)
        plt.imshow(rgb_frame)
        plt.axis('off')
        plt.show()

        time.sleep(0.1)

except KeyboardInterrupt:
    print("Interrupted by user")

finally:
    cap.release()
    # Avoid cv2.destroyAllWindows() to prevent errors in some environments

# After loop ends, print final result below the cell
if authenticated:
    print(f"\n✅ Authentication successful for user: {identity}")
else:
    print(f"\n❌ Authentication failed for user: {identity}")

# Optionally show the last frame with the label
if last_frame is not None:
    rgb_last_frame = cv2.cvtColor(last_frame, cv2.COLOR_BGR2RGB)
    plt.imshow(rgb_last_frame)
    plt.axis('off')
    plt.title("Last captured frame")
    plt.show()










               


                   
            




# -------------------------------Admin Panel------------------------------

## - Facilitates Create, Read, Update, Delete operations

## - Demonstrates User Authentication 

In [1]:
# ========== Imports ==========
import os, time, cv2, pickle
import numpy as np
import face_recognition
from scipy.io.wavfile import write, read
import matplotlib.pyplot as plt
import sounddevice as sd
import pickle
import python_speech_features
from scipy.io import wavfile
from IPython.display import display, clear_output
from ipywidgets import VBox, HBox, Button, Dropdown, Text, Output, Label
from sklearn.mixture import GaussianMixture

# ========== Paths ==========
image_path = "./captured_images"
voice_data_path = "./voice_database"

# ========== Widgets ==========
output = Output()
new_user_text = Text(placeholder='Enter new username...')
user_dropdown = Dropdown(description='Users:')
add_user_button = Button(description="➕ Add")
delete_user_button = Button(description="❌ Delete")
update_user_button = Button(description="🔄 Update")
view_user_button = Button(description="👁️ View")
refresh_button = Button(description="🔁 Refresh")
authenticate_button = Button(description="🔐 Authenticate")

# ========== Utilities ==========

def countdown(seconds):
    for i in range(seconds, 0, -1):
        print(f"⏳ Recording in {i}...")
        time.sleep(1)

def get_user_list():
    return sorted(os.listdir(image_path)) if os.path.exists(image_path) else []

def save_user_face_encoding(user_folder_path):
    image_files = [f for f in os.listdir(user_folder_path) if f.lower().endswith(('.jpg', '.png'))]
    encodings = []
    for image_file in image_files:
        img_path = os.path.join(user_folder_path, image_file)
        img = face_recognition.load_image_file(img_path)
        encoding = face_recognition.face_encodings(img)
        if encoding:
            encodings.append(encoding[0])
    if encodings:
        avg = np.mean(encodings, axis=0)
        with open(os.path.join(user_folder_path, 'face_encoding.pkl'), 'wb') as f:
            pickle.dump(avg, f)
        print("✅ Face encoding saved.")
    else:
        print("❌ No valid encodings found.")

def register_voice(name):
    user_voice_path = os.path.join(voice_data_path, name)
    os.makedirs(user_voice_path, exist_ok=True)
    for i in range(1, 4):
        print(f"🎙️ Say 'hello computer' sample {i}/3")
        countdown(3)
        fs = 44100
        duration = 3
        voice = sd.rec(int(duration * fs), samplerate=fs, channels=1)
        sd.wait()
        file_path = os.path.join(user_voice_path, f"hello_computer_{i}.wav")
        write(file_path, fs, voice)
        print(f"✅ Saved voice to {file_path}")

def compare_voices(file1, file2):
    try:
        _, data1 = read(file1)
        _, data2 = read(file2)
        min_len = min(len(data1), len(data2))
        if min_len == 0: return 1.0
        return np.linalg.norm(data1[:min_len] - data2[:min_len]) / min_len
    except:
        return 1.0

def recognize_voice(selected_user):
    
    """Authenticate user using GMM model comparison"""
    # Record new voice sample
    fs = 44100
    duration = 3
    countdown(3)
    print("🔴 Recording...")
    voice_data = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    temp_path = "./temp_voice.wav"
    write(temp_path, fs, voice_data)

    # Load the target user's GMM model
    gmm_path = f"./gmm_models/{selected_user}.gmm"
    if not os.path.exists(gmm_path):
        print(f"❌ No GMM model found for {selected_user}")
        os.remove(temp_path)
        return False

    try:
        # Extract features
        (rate, sig) = wavfile.read(temp_path)
        mfcc_feat = python_speech_features.mfcc(
            sig, rate, 
            winlen=0.025, winstep=0.01, numcep=13,
            nfilt=26, nfft=2048, lowfreq=0, 
            highfreq=None, preemph=0.97,
            ceplifter=22, appendEnergy=True
        )

        # Load GMM model
        with open(gmm_path, 'rb') as gmm_file:
            gmm = pickle.load(gmm_file)

        # Calculate log likelihood
        scores = gmm.score(mfcc_feat)
        log_likelihood = scores.sum()
        print(f"🔍 Voice match score for {selected_user}: {log_likelihood:.2f}")

        # Threshold for acceptance (adjust based on your testing)
        threshold = -60  # Typical range: -10 to -30
        is_match = log_likelihood > threshold
        
        # Clean up
        os.remove(temp_path)
        
        return is_match

    except Exception as e:
        print(f"⚠️ Error during voice recognition: {str(e)}")
        if os.path.exists(temp_path):
            os.remove(temp_path)
        return False
        

def load_all_face_encodings(base_path='captured_images'):
    known_face_encodings = []
    known_face_names = []
    if not os.path.exists(base_path): return known_face_encodings, known_face_names
    for user_name in os.listdir(base_path):
        pkl_path = os.path.join(base_path, user_name, 'face_encoding.pkl')
        if os.path.exists(pkl_path):
            with open(pkl_path, 'rb') as f:
                known_face_encodings.append(pickle.load(f))
                known_face_names.append(user_name)
    return known_face_encodings, known_face_names

# ========== Button Functions ==========

def on_add_user(b):
    with output:
        clear_output()
        name = new_user_text.value.strip()
        if not name:
            print("❌ Please enter a name.")
            return
        print(f"👤 Registering {name}")
        img_dir = os.path.join(image_path, name)
        os.makedirs(img_dir, exist_ok=True)

        cap = cv2.VideoCapture(0)
        for i in range(3):
            ret, frame = cap.read()
            if ret:
                file_path = os.path.join(img_dir, f"{name}_{i+1}.jpg")
                cv2.imwrite(file_path, frame)
                print(f"📷 Image {i+1}/3 saved.")
                time.sleep(1)
        cap.release()

        register_voice(name)
        save_user_face_encoding(img_dir)
        train_gmm_model(name)
        refresh_user_list()
        print("✅ User registered.")

def on_delete_user(b):
    with output:
        clear_output()
        name = user_dropdown.value
        if not name:
            print("❌ Select user to delete.")
            return
        img_dir = os.path.join(image_path, name)
        voice_dir = os.path.join(voice_data_path, name)
        for path in [img_dir, voice_dir]:
            if os.path.exists(path):
                for f in os.listdir(path):
                    os.remove(os.path.join(path, f))
                os.rmdir(path)
        refresh_user_list()
        print(f"🗑️ Deleted {name}.")

from IPython.display import Audio

def on_view_user(b):
    with output:
        clear_output()
        name = user_dropdown.value
        if not name:
            print("❌ No user selected.")
            return

        print(f"👁️ Viewing data for: {name}")
        
        # === Show face images ===
        img_dir = os.path.join(image_path, name)
        if os.path.exists(img_dir):
            image_files = sorted([f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.png'))])
            if image_files:
                print("🖼️ Face Images:")
                for file in image_files:
                    img = cv2.imread(os.path.join(img_dir, file))
                    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    plt.imshow(rgb)
                    plt.axis('off')
                    plt.title(file)
                    plt.show()
            else:
                print("⚠️ No face images found.")
        else:
            print("❌ Face image folder not found.")

        # === Show voice samples ===
        voice_dir = os.path.join(voice_data_path, name)
        if os.path.exists(voice_dir):
            wav_files = sorted([f for f in os.listdir(voice_dir) if f.lower().endswith('.wav')])
            if wav_files:
                print("\n🎧 Voice Samples:")
                for file in wav_files:
                    full_path = os.path.join(voice_dir, file)
                    print(f"🔊 {file}")
                    display(Audio(full_path, autoplay=False))
            else:
                print("⚠️ No voice samples found.")
        else:
            print("❌ Voice data folder not found.")


def on_update_user(b):
    with output:
        clear_output()

        name = user_dropdown.value
        if not name:
            print("❌ Please select a user to update.")
            return

        print(f"🔧 Updating user: {name}")

        # Re-capture images
        user_image_dir = os.path.join(image_path, name)
        os.makedirs(user_image_dir, exist_ok=True)

        cap = cv2.VideoCapture(0)
        count = 0
        while count < 3:
            ret, frame = cap.read()
            if not ret:
                print("❌ Failed to capture image from camera.")
                break
            img_path = os.path.join(user_image_dir, f"{name}_{count+1}.jpg")
            cv2.imwrite(img_path, frame)
            print("📸 Image updated.")
            count += 1
            time.sleep(1)
        cap.release()

        # Save face encoding
        save_user_face_encoding(user_image_dir)
        

        # Re-record voice samples (3 times with countdown)
        user_voice_dir = os.path.join(voice_data_path, name)
        os.makedirs(user_voice_dir, exist_ok=True)

        fs = 44100
        duration = 3
        for i in range(3):
            print(f"\n🎙️ Preparing voice sample {i+1}/3...")
            for j in reversed(range(1, 4)):
                print(f"⏳ Recording in {j}...")
                time.sleep(1)
            print("🔴 Recording...")
            voice_data = sd.rec(int(duration * fs), samplerate=fs, channels=1)
            sd.wait()
            voice_path = os.path.join(user_voice_dir, f"hello_computer_{i+1}.wav")
            write(voice_path, fs, voice_data)
            print(f"✅ Saved: {voice_path}")

            train_gmm_model(name)
        print(f"\n✅ Update complete for user: {name}")


def refresh_user_list(b=None):
    output.clear_output()  # Clears the output area

    # Clear the dropdown and text field
    user_dropdown.options = get_user_list()
    user_dropdown.value = None
    new_user_text.value = ''
    

def train_gmm_model(user_name, n_components=16):
    
    voice_dir = os.path.join(voice_data_path, user_name)
    features = []

    for file in os.listdir(voice_dir):
        if file.endswith(".wav"):
            filepath = os.path.join(voice_dir, file)
            try:
                (rate, sig) = wavfile.read(filepath)
                mfcc_feat = python_speech_features.mfcc(
                    sig, rate, 
                    winlen=0.025, winstep=0.01, numcep=13,
                    nfilt=26, nfft=2048, lowfreq=0,
                    highfreq=None, preemph=0.97,
                    ceplifter=22, appendEnergy=True
                )
                features.append(mfcc_feat)
            except Exception as e:
                print(f"⚠️ Error processing {filepath}: {e}")

    if not features:
        print(f"❌ No valid voice files to train GMM for {user_name}")
        return

    X = np.vstack(features)
    gmm = GaussianMixture(n_components=n_components, covariance_type='diag', n_init=3)
    gmm.fit(X)

    os.makedirs("./gmm_models", exist_ok=True)
    with open(f"./gmm_models/{user_name}.gmm", 'wb') as gmm_file:
        pickle.dump(gmm, gmm_file)
    print(f"✅ GMM model saved for {user_name}")



def authenticate_user(b):
    with output:
        clear_output()
        selected_user = user_dropdown.value
        if not selected_user:
            print("❌ Please select a user from the dropdown.")
            return
        print(f"🎙️ Starting authentication for: {selected_user}")
        known_face_encodings, known_face_names = load_all_face_encodings()

        if selected_user not in known_face_names:
            print(f"❌ No face encoding found for {selected_user}.")
            return

        voice_match = recognize_voice(selected_user)
        if not voice_match:
            print("❌ Voice authentication failed.")
            return
        print("✅ Voice matched. Please look at the camera...")

        cap = cv2.VideoCapture(0)
        cap.set(3, 640)
        cap.set(4, 480)
        time.sleep(1)
        authenticated = False
        last_frame = None
        timeout = time.time() + 20

        while time.time() < timeout:
            ret, frame = cap.read()
            if not ret: continue
            small = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
            locs = face_recognition.face_locations(rgb)
            if len(locs) == 1:
                enc = face_recognition.face_encodings(rgb, locs)[0]
                idx = known_face_names.index(selected_user)
                dist = face_recognition.face_distance([known_face_encodings[idx]], enc)[0]
                top, right, bottom, left = [v * 4 for v in locs[0]]
                color = (0, 255, 0) if dist < 0.5 else (0, 0, 255)
                label = f"{selected_user} ✅" if dist < 0.5 else f"{selected_user} ❌"
                cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
                cv2.putText(frame, label, (left, bottom - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                authenticated = dist < 0.5
                last_frame = frame.copy()
                if authenticated:
                    break
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            clear_output(wait=True)
            plt.imshow(rgb_frame)
            plt.axis("off")
            plt.show()
        cap.release()
        clear_output(wait=True)
        if authenticated:
            print(f"✅ Authentication successful for {selected_user}")
        else:
            print(f"❌ Authentication failed for {selected_user}")
        if last_frame is not None:
            plt.imshow(cv2.cvtColor(last_frame, cv2.COLOR_BGR2RGB))
            plt.axis("off")
            plt.title("Final Frame")
            plt.show()

    
# ========== Button Bindings ==========
add_user_button.on_click(on_add_user)
delete_user_button.on_click(on_delete_user)
update_user_button.on_click(on_update_user)
view_user_button.on_click(on_view_user)
refresh_button.on_click(refresh_user_list)
authenticate_button.on_click(authenticate_user)


# ========== Admin UI ==========
admin_panel = VBox([
    Label("🔐 Admin Panel for Biometric Authentication System"),
    HBox([new_user_text, add_user_button, delete_user_button, update_user_button]),
    HBox([user_dropdown, view_user_button, authenticate_button, refresh_button]),
    output
])

display(admin_panel)
refresh_user_list()


In [ ]:
%whos function
